# Recipe Recommender System

**Want a specific meal for within a certain budget for a specific number of people**

In [1]:
import pandas as pd
import numpy as np
import pickle

## Read in Files

### Grocery Similarity Model

In [2]:
grocery_similarities = pd.read_csv('../../data/05_model_output/data_matrix_sim.csv')

In [ ]:
grocery_similarities.set_index('Unnamed: 0', inplace=True)

### Recipe Database

In [3]:
infile = open('../../data/03_processed/crf_ingred_dict','rb')
recipe_dict = pickle.load(infile)
infile.close()

In [4]:
infile = open('../../data/03_processed/crf_links_dict','rb')
recipe_links_dict = pickle.load(infile)
infile.close()

### Grocery Prices 

In [33]:
grocery_prices_marianos = pd.read_csv('../../data/03_processed/grocery_prices_marianos_final.csv')

***

In [7]:
def extract_ingredients(single_recipe_ing_list):
    ret = []
    for ingredient in single_recipe_ing_list:
        ret.append(ingredient['name'])
    return ret 

In [8]:
def find_recipe_by_ingredient(recipe_dict, search_item):
    
    listofkeys = []
    listofitems = recipe_dict.items()

    for item in listofitems:
        lst = extract_ingredients(item[1])
        subs = search_item
        if any(subs in s for s in lst):
            listofkeys.append(item[0])
    return listofkeys

In [32]:
print(grocery_similarities.loc['blueberries'].nlargest(11))

blueberries     1.000000
strawberries    0.232235
raspberries     0.195576
spinach         0.152607
avocado         0.145650
banana          0.144528
milk            0.144380
blackberries    0.141411
bananas         0.140091
yogurt          0.138631
cheese          0.132716
Name: blueberries, dtype: float64


In [23]:
find_recipe_by_ingredient(recipe_dict, 'avocado')

['hawaiian spam tacos with pineapple',
 'chorizo and egg breakfast tacos',
 'slow cooker mexican pulled pork tacos',
 'pressure cooker mexican pulled pork (carnitas)',
 'pozole rojo (mexican pork and hominy stew)',
 'sheet pan chicken fajitas with peppers and onions',
 'chicken tortilla soup',
 'slow cooker shredded chicken chili',
 'chicken and black bean tostadas',
 'pressure cooker shredded chicken taco meat',
 'chicken and avocado salad with buttermilk dressing',
 'spicy grilled chicken tacos',
 'grilled chicken nachos',
 'slow cooker shredded chicken taco meat',
 'chicken pozole salad',
 'grilled lime chicken with black bean sauce',
 'crispy chicken skin tacos with habanero salsa',
 'chicken pozole',
 'skillet shrimp fajitas',
 'broccoli and cheddar quesadillas',
 'skirt steak with avocado chimichurri',
 'easy avocado and black bean quesadillas',
 'edamame avocado dip',
 'green goddess deviled eggs',
 'loaded sheet pan nachos',
 '7 layer bean dip',
 'buffalo cauliflower tacos with

In [22]:
recipe_dict['moroccan chickpea barley salad']

[{'qty': '1', 'unit': '1/2  cups', 'name': 'barley', 'comment': ''},
 {'qty': '1',
  'unit': '1/2  cups',
  'name': 'chicken',
  'comment': 'or veggie broth use veg broth for vegan version'},
 {'qty': '1', 'unit': '1/2  cups', 'name': 'water', 'comment': ''},
 {'qty': '1', 'unit': 'teaspoon', 'name': 'salt', 'comment': ''},
 {'qty': 'Extra', 'unit': '', 'name': 'virgin olive oil', 'comment': ''},
 {'qty': '1',
  'unit': '15 ounce',
  'name': 'can of chickpeas garbanzo beans',
  'comment': ''},
 {'qty': 'rinsed', 'unit': '', 'name': 'and strained', 'comment': ''},
 {'qty': 'or', 'unit': '', 'name': '1 1/2  cups', 'comment': ''},
 {'qty': 'cooked', 'unit': '', 'name': '', 'comment': ''},
 {'qty': '1', 'unit': 'cup', 'name': 'shelled pistachio nuts', 'comment': ''},
 {'qty': '1', 'unit': 'cup', 'name': 'diced dried apricots', 'comment': ''},
 {'qty': '2',
  'unit': '',
  'name': '3 chopped green onions',
  'comment': 'or scallions'},
 {'qty': '1', 'unit': 'cup', 'name': 'chopped parsley',

## Recommendor System sudo-code

```python


```